In [ ]:
# ! pip install transformers
# !pip install transformers datasets sentencepiece seqeval
! pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.6/312.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
import pandas as pd
data = pd.read_excel("/content/drive/MyDrive/data/Raw_data.xlsx")
data.head()

,Sentence #,Word,Tag
0,Sentence: 1,1)पालिकेचे,O
1,NaN,नाव,O
2,NaN,मुंबई,BLOC
3,NaN,मनपाइतर,O
4,NaN,वर्णन,O


In [ ]:
data.dropna(subset=['Word'], how='all', inplace=True)
data

,Sentence #,Word,Tag
0,Sentence: 1,1)पालिकेचे,O
1,NaN,नाव,O
2,NaN,मुंबई,BLOC
3,NaN,मनपाइतर,O
4,NaN,वर्णन,O
...,...,...,...
13138,NaN,195,BCTS
13139,NaN,part,ICTS
13140,NaN,;,O
13142,NaN,),O


In [ ]:
# pandas has a very handy "forward fill" function to fill missing values based on the last upper non-nan value
data = data.fillna(method='ffill')
data.head()

,Sentence #,Word,Tag
0,Sentence: 1,1)पालिकेचे,O
1,Sentence: 1,नाव,O
2,Sentence: 1,मुंबई,BLOC
3,Sentence: 1,मनपाइतर,O
4,Sentence: 1,वर्णन,O


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data['Sentence #'] = LabelEncoder().fit_transform(data['Sentence #'])

In [ ]:
data.head(30)

,Sentence #,Word,Tag
0,0,1)पालिकेचे,O
1,0,नाव,O
2,0,मुंबई,BLOC
3,0,मनपाइतर,O
4,0,वर्णन,O
6,0,सदनिका,O
7,0,नं,O
9,0,शॉप,O
10,0,नं,O
11,0,70,BUNITNO


In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace=True)

In [ ]:
data["labels"] = data["labels"].str.upper()

In [ ]:
x = data[["sentence_id","words"]]
y = data["labels"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)

In [ ]:
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"], "words":x_train["words"], "labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"], "words":x_test["words"], "labels":y_test})

Model Training

In [ ]:
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
label = data["labels"].unique().tolist()
label

['O',
 'BLOC',
 'BUNITNO',
 'BFLOOR',
 'IFLOOR',
 'BNAME',
 'INAME',
 'ILOC',
 'BCTS',
 'ICTS',
 'IUNITNO']

In [ ]:
args = NERArgs()
args.num_train_epochs = 20
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 10
args.eval_batch_size = 10
args.max_len = 500

In [ ]:
model = NERModel('bert','bert-base-cased', labels = label, args = args,use_cuda=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
model.train_model(train_data, eavl_data = test_data, acc = accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/20 [00:00<?, ?it/s]

...

Running Epoch 17 of 20:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/20 [00:00<?, ?it/s]

(400, 0.2993944656243548)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: IFLOOR seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: INAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ICTS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: BCTS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: BLOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:

In [ ]:
result

{'eval_loss': 0.9688526093959808,
 'precision': 0.5696649029982364,
 'recall': 0.6247582205029013,
 'f1_score': 0.595940959409594}

In [ ]:
prediction, model_outputs = model.predict(["पालिकेचे नाव:मुंबई म.न.पा.इतर वर्णन :सदनिका नं: 507, माळा नं: 5,बील्डींग नं.3, इमारतीचे नाव: त्रिमूर्ति सीएचएस लिमिटेड,एमएमआरडीए कॉलनी,, ब्लॉक नं: स्टेशन रोड कांजुर मार्ग् पश्चिम, रोड नं: मुंबई-400078(  (     C.T.S. Number : 586,587,591 ;  )  )"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'पालिकेचे': 'O'},
  {'नाव:मुंबई': 'O'},
  {'म.न.पा.इतर': 'O'},
  {'वर्णन': 'O'},
  {':सदनिका': 'O'},
  {'नं:': 'O'},
  {'507,': 'BUNITNO'},
  {'माळा': 'O'},
  {'नं:': 'O'},
  {'5,बील्डींग': 'BFLOOR'},
  {'नं.3,': 'O'},
  {'इमारतीचे': 'O'},
  {'नाव:': 'O'},
  {'त्रिमूर्ति': 'BLOC'},
  {'सीएचएस': 'BLOC'},
  {'लिमिटेड,एमएमआरडीए': 'BLOC'},
  {'कॉलनी,,': 'O'},
  {'ब्लॉक': 'O'},
  {'नं:': 'O'},
  {'स्टेशन': 'BLOC'},
  {'रोड': 'O'},
  {'कांजुर': 'O'},
  {'मार्ग्': 'O'},
  {'पश्चिम,': 'ILOC'},
  {'रोड': 'O'},
  {'नं:': 'O'},
  {'मुंबई-400078(': 'BLOC'},
  {'(': 'O'},
  {'C.T.S.': 'O'},
  {'Number': 'O'},
  {':': 'O'},
  {'586,587,591': 'O'},
  {';': 'O'},
  {')': 'O'},
  {')': 'O'}]]

In [ ]:
prediction, model_outputs = model.predict(["1)पालिकेचे नाव:Mumbai Ma.na.pa. इतर वर्णन :सदनिका नं: 1906,ए विंग, माळा नं: 19, इमारतीचे नाव: गौरी एक्सलेंसी, रोड : बाभरेकर नगर,कांदिवली पश्चिम मुंबई 400067(  (     C.T.S. Number : 470 P , 471A P ;  )  )"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'1)पालिकेचे': 'O'},
  {'नाव:Mumbai': 'O'},
  {'Ma.na.pa.': 'O'},
  {'इतर': 'INAME'},
  {'वर्णन': 'O'},
  {':सदनिका': 'O'},
  {'नं:': 'O'},
  {'1906,ए': 'O'},
  {'विंग,': 'O'},
  {'माळा': 'O'},
  {'नं:': 'O'},
  {'19,': 'O'},
  {'इमारतीचे': 'O'},
  {'नाव:': 'O'},
  {'गौरी': 'BLOC'},
  {'एक्सलेंसी,': 'O'},
  {'रोड': 'O'},
  {':': 'O'},
  {'बाभरेकर': 'O'},
  {'नगर,कांदिवली': 'INAME'},
  {'पश्चिम': 'ILOC'},
  {'मुंबई': 'BLOC'},
  {'400067(': 'O'},
  {'(': 'O'},
  {'C.T.S.': 'O'},
  {'Number': 'O'},
  {':': 'O'},
  {'470': 'BUNITNO'},
  {'P': 'O'},
  {',': 'O'},
  {'471A': 'BCTS'},
  {'P': 'O'},
  {';': 'O'},
  {')': 'O'},
  {')': 'O'}]]

In [ ]:
prediction, model_outputs = model.predict(["1)पालिकेचे नाव:मुंबई मनपाइतर वर्णन :सदनिका नं: सदनिका क्र 502, माळा नं: 5  वा माळा बी वींग , इमारतीचे नाव: मधुपुरी अपार्टमेंट्स , ब्लॉक नं: कांदिवली प, रोड : गोखले रोड डहाणूकर वाडी , इतर  माहिती:  सोबत 1 कार पार्किंग स्पेस सहित, सदनिकाचा  क्षेत्रफळ 759 चौ फूट कार्पेट(  (     C.T.S. Number : 1017 ;  )  )"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'1)पालिकेचे': 'O'},
  {'नाव:मुंबई': 'O'},
  {'मनपाइतर': 'O'},
  {'वर्णन': 'O'},
  {':सदनिका': 'O'},
  {'नं:': 'O'},
  {'सदनिका': 'O'},
  {'क्र': 'O'},
  {'502,': 'BUNITNO'},
  {'माळा': 'O'},
  {'नं:': 'O'},
  {'5': 'BFLOOR'},
  {'वा': 'IFLOOR'},
  {'माळा': 'O'},
  {'बी': 'O'},
  {'वींग': 'O'},
  {',': 'O'},
  {'इमारतीचे': 'O'},
  {'नाव:': 'O'},
  {'मधुपुरी': 'O'},
  {'अपार्टमेंट्स': 'O'},
  {',': 'O'},
  {'ब्लॉक': 'O'},
  {'नं:': 'O'},
  {'कांदिवली': 'O'},
  {'प,': 'ILOC'},
  {'रोड': 'O'},
  {':': 'O'},
  {'गोखले': 'O'},
  {'रोड': 'O'},
  {'डहाणूकर': 'O'},
  {'वाडी': 'O'},
  {',': 'O'},
  {'इतर': 'O'},
  {'माहिती:': 'O'},
  {'सोबत': 'O'},
  {'1': 'BCTS'},
  {'कार': 'O'},
  {'पार्किंग': 'O'},
  {'स्पेस': 'O'},
  {'सहित,': 'O'}]]

In [ ]:
prediction, model_outputs = model.predict(["1)पालिकेचे नाव:मुंबई मनपाइतर वर्णन :सदनिका नं: ११०१ , माळा नं: ११ वा माळा,बी विंग, इमारतीचे नाव: मधुपुरी अपार्टमेंट्स  , ब्लॉक नं: कांदिवली प 67, रोड : गोखले रोड डहाणूकर वाडी , इतर  माहिती: सोबत १ कार पार्किंग स्पेस सहित, सदनिकाचा  क्षेत्रफळ ९२० चौ फूट कार्पेट(  (     C.T.S. Number : 1017 ;  )  )"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'1)पालिकेचे': 'O'},
  {'नाव:मुंबई': 'O'},
  {'मनपाइतर': 'BNAME'},
  {'वर्णन': 'O'},
  {':सदनिका': 'O'},
  {'नं:': 'O'},
  {'११०१': 'BLOC'},
  {',': 'O'},
  {'माळा': 'O'},
  {'नं:': 'O'},
  {'११': 'BLOC'},
  {'वा': 'IFLOOR'},
  {'माळा,बी': 'O'},
  {'विंग,': 'O'},
  {'इमारतीचे': 'O'},
  {'नाव:': 'O'},
  {'मधुपुरी': 'O'},
  {'अपार्टमेंट्स': 'O'},
  {',': 'O'},
  {'ब्लॉक': 'O'},
  {'नं:': 'O'},
  {'कांदिवली': 'O'},
  {'प': 'ILOC'},
  {'67,': 'O'},
  {'रोड': 'O'},
  {':': 'O'},
  {'गोखले': 'BNAME'},
  {'रोड': 'O'},
  {'डहाणूकर': 'O'},
  {'वाडी': 'O'},
  {',': 'O'},
  {'इतर': 'O'},
  {'माहिती:': 'O'},
  {'सोबत': 'O'},
  {'१': 'BLOC'},
  {'कार': 'O'},
  {'पार्किंग': 'O'},
  {'स्पेस': 'O'},
  {'सहित,': 'O'},
  {'सदनिकाचा': 'O'}]]

In [ ]:
# import pandas as pd

In [ ]:
# test_2 = pd.read_excel("/content/drive/MyDrive/data/test_data_10.xlsx")

In [ ]:
# prediction, model_outputs = model.predict(test_2)

In [ ]:
# prediction

In [ ]:
# import pickle

# pickle.dump(model, open('BERT_Model.pkl','wb'))
# Finalmodel=pickle.load(open('BERT_Model.pkl','rb'))